In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Documents

In [3]:
# Additional imports for loading PDF documents and QA chain.
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [4]:
# Create the PDF loader and load the document.
pdf_loader = PyPDFLoader('https://static.bc-edx.com/ai/ail-v-1-0/m22/lesson_1/datasets/Resume.pdf')
documents = pdf_loader.load()

# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.0)


# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
query = 'Could I write to Jane Doe in Spanish and expect her to understand?'

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents": documents, "question": query})
print(result["output_text"])

C:\Users\metin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(


Based on the provided information, Jane Doe lists **Spanish (Conversational)** as one of her languages. This means she likely understands spoken Spanish, but it's not clear how proficient she is in written Spanish. 

It's possible she could understand a simple message, but it's not guaranteed. If you need to communicate something important, it's best to use English. 



# Built-in Integrations

In [5]:
# Additional imports for loading Wikipedia content and QA chain.
from langchain_community.document_loaders import WikipediaLoader
from langchain.chains.question_answering import load_qa_chain

In [6]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.0)

# Define the wikipedia topic as a string.
wiki_topic = "Leonard Cohen"

# Load the wikipedia results as documents, using a max of 2.
documents = WikipediaLoader(query=wiki_topic, load_max_docs=2, load_all_available_meta=True).load()

# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
query = 'How many albums has Leonard Cohen released?'

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents": documents, "question": query})
print(result["output_text"])

Leonard Cohen released **15 studio albums** in his lifetime. 



# External APIs

In [7]:
# Additional imports for API chain.
from langchain.chains import APIChain

In [8]:
# Text description of API spec.
spec = """URL STRUCTURE
        Go to URL https://opentdb.com/api.php to get a plain text response, where
        the category is of type Computers, Mathematics, or Mythology,
        which are mapped to the numbers 18, 19, and 20 respectively.
        The amount parameter has to be specified to determine how many questions to return.
        The difficulty can be set to easy, medium, or hard. 
        Include no other text, only the API call URL. Don't use newlines."""

# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.9)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm,
    api_docs=spec,
    verbose=True,
    limit_to_domains=["https://opentdb.com/api.php"]
)

# Define a query as a dictionary.
query = {"question": "What is a difficult trivia question about Computers?"}

# Run the chain using the query, and print the response.
result = chain.invoke(query)
print(result['output'])



> Entering new APIChain chain...
https://opentdb.com/api.php?amount=1&category=18&difficulty=hard 

{"response_code":0,"results":[{"type":"multiple","difficulty":"hard","category":"Science: Computers","question":"Which of the following computer components can be built using only NAND gates?","correct_answer":"ALU","incorrect_answers":["CPU","RAM","Register"]}]}

> Finished chain.
A difficult trivia question about Computers is: "Which of the following computer components can be built using only NAND gates?". The answer is ALU. 



In [9]:
# Additional imports for API chain and processing json.
from langchain.chains import APIChain
import json
import requests

In [10]:
# Store the API key in a variable.
api_key = os.getenv("NASA_API_KEY")

# Text description of API spec.
spec = """URL STRUCTURE
        To retrieve data on Geomagnetic Storms (GSTs) using the NASA API,         
        access the endpoint at https://api.nasa.gov/DONKI/GST. 
        Specify the start date using the "startDate" parameter in the format "yyyy-MM-dd"
        and the end date using the "endDate" parameter in the same format. 
        Ensure to include your API key using the "api_key" parameter, for example, "api_key=DEMO_KEY.
         Include no other text, only the API call URL. Don't use newlines.
        """

# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.0)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm,
    api_docs=json.dumps(spec),
    verbose=False,  # Changing to True will expose user's NASA_API_KEY API key.
    limit_to_domains=["https://api.nasa.gov/DONKI/GST"],
)

# Define a query as a string.
query = {"question": 'Get a list of the 3 strongest Geomagnetic Storms in 2024 and show the details such as when they occurred and the corresponding KP Index for each one of them?'}

# Run the chain using the query, and print the response.
result = chain.invoke(query)

print(result["output"])

Summary:

The 3 strongest Geomagnetic Storms in 2024 were:

1. **2024-05-10T15:00:00-GST-001**: Occurred on May 10th, 2024, with a peak KP Index of 9.0.
2. **2024-08-12T00:00:00-GST-001**: Occurred on August 12th, 2024, with a peak KP Index of 8.0.
3. **2024-03-24T12:00:00-GST-001**: Occurred on March 24th, 2024, with a peak KP Index of 8.0. 

